In [1]:
import ngsolve as ngs
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw
from ngsolve.krylovspace import CGSolver
from ttictoc import tic, toc

mesh = ngs.Mesh('whatever.vol')
mesh.Curve(2)
# mesh.Refine()

In [2]:
%run J.ipynb

using 3d mesh with ne= 189675  elements and nv= 32708  points
Solving...
... Success!
Number of iterations = 506/1000 | Residual = 9.18203065927305e-06
2999.9999999999877
Solving...
... Success!
Number of iterations = 516/1000 | Residual = 9.8644439577931e-06
2999.9998732485515


In [3]:
from bhdata import BHCurves
fun_dw  = BHCurves(4)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil|ambient|default"
nonlinear = "r_steel|l_steel|mid_steel"

BH data from TEAM 13 problem
returning energy


In [4]:
# print(HCurl.ndof)
# print(mesh.GetMaterials())

p = 2

HCurl = ngs.HCurl(mesh, order = p, nograds = True)
u,v = HCurl.TnT()


# Nonlinear:

maxit = 10000
tol2 = 1e-8
regb = 1e-12

A = ngs.GridFunction(HCurl)
B = ngs.curl(A)
normB = ngs.sqrt(B*B + regb)



cf_energy = mesh.MaterialCF({linear: nu0/2*B*B, nonlinear: fun_w(normB)}, default = nu0/2*B*B)
def fun_W():
    with ngs.TaskManager(): res = ngs.Integrate(cf_energy - Hs*ngs.curl(A), mesh)
    return res



cf_rhs = mesh.MaterialCF({linear: nu0, nonlinear: fun_dw(normB)/normB}, default = nu0)
# Draw(mu_nl, mesh)
# rhs = ngs.LinearForm(- (Hs*ngs.curl(v))*ngs.dx)
# rhs = ngs.LinearForm(- (ngs.curl(Hs)*v)*ngs.dx)
rhs = ngs.LinearForm((cf_rhs*ngs.curl(A)*ngs.curl(v) - Hs*ngs.curl(v))*ngs.dx)

def fun_dW(): #implicitly depending on A!
    with ngs.TaskManager(): rhs.Assemble()
    return rhs



Id = ngs.CF((1,0,0,
             0,1,0,
             0,0,1), dims=(3,3))

BBt = ngs.CF((B[0]*B[0], B[0]*B[1], B[0]*B[2],
              B[1]*B[0], B[1]*B[1], B[1]*B[2],
              B[2]*B[0], B[2]*B[1], B[2]*B[2]), dims =(3,3))

fun1 = fun_dw(normB)/normB
fun2 = (fun_ddw(normB) - fun_dw(normB)/normB)/(normB*normB)

cf_iter = mesh.MaterialCF({linear: nu0*Id, nonlinear: fun1*Id + fun2*BBt}, default = nu0*Id)

K_iter = ngs.BilinearForm(HCurl, nograds = True)
K_iter += (cf_iter*ngs.curl(u)*ngs.curl(v))*ngs.dx
C_iter = ngs.Preconditioner(K_iter, type = "local")

def fun_ddW():
    with ngs.TaskManager(): K_iter.Assemble()
    return K_iter


In [5]:
print("Using 3D mesh with ne=", mesh.ne, "elements and nv=", mesh.nv, "points and " ,HCurl.ndof, "DOFs.\n ")

with ngs.TaskManager(): A.Set(ngs.CF((0,0,0)))

du = ngs.GridFunction(HCurl)
uo = ngs.GridFunction(HCurl)
wo = 1e12

for it in range(1,maxit+1):
    tic()
    # with ngs.TaskManager():
    #     K_iter.Assemble()
    #     rhs.Assemble()
    #     res = ngs.Integrate(cf_energy - Hs*ngs.curl(A), mesh)
    
    w  = fun_W()
    dw = fun_dW()
    da = fun_ddW()
    tm1 = toc()
    
    tic()
    # du.vec.data = da.mat.Inverse(HCurl.FreeDofs(), inverse="sparsecholesky") * dw.vec 
    # iterativeSolver = CGSolver(K_iter.mat, freedofs = HCurl.FreeDofs(), atol = 1e-2,  maxiter = maxit, printrates = False)
    with ngs.TaskManager():
        iterativeSolver = CGSolver(K_iter.mat, pre = C_iter.mat, tol  = 1e-2,  maxiter = maxit)
        du.vec.data = iterativeSolver * dw.vec 
    
    if len(iterativeSolver.residuals) == maxit: print("... Success!")
    # print(f"Number of iterations = {iterativeSolver.iterations}/{maxit} | Residual = {iterativeSolver.residuals[-1]}")
    tm2 = toc()

    nrm = ngs.InnerProduct(du.vec,dw.vec)
    
    if it == 1:
        nrm0 = nrm

    if nrm/nrm0<tol2:
        print("converged to desired tolerance")
        break
    elif abs(wo-w) < tol2*1e-2:
        print("stopped early due to stagnation")
        break
    else:
        # linesearch
        uo.vec.data = A.vec.data
        wo = w
        alpha = 1
        for init in range(1,21):
            A.vec.data -= alpha*du.vec.data
            wn = fun_W()
            if wn < w - alpha*0.1*nrm:
                print("Iter: %2d | assem : %.2fs | CG took %.2fs with %4d iterations | alpha : %.2f | energy = %.10f | relres = %.2e |"  %(it,tm1,tm2,iterativeSolver.iterations,alpha,w,nrm/nrm0))
                break
            else:
                alpha = alpha/2
                A.vec.data = uo.vec.data

Using 3D mesh with ne= 189675 elements and nv= 32708 points and  982264 DOFs.
 
Iter:  1 | assem : 3.70s | CG took 2.73s with  177 iterations | alpha : 1.00 | energy = 0.0000000000 | relres = 1.00e+00 |
Iter:  2 | assem : 3.70s | CG took 5.19s with  323 iterations | alpha : 0.25 | energy = -1.2710148664 | relres = 9.08e-01 |
Iter:  3 | assem : 3.96s | CG took 2.74s with  167 iterations | alpha : 0.50 | energy = -1.6327543004 | relres = 1.54e-02 |
Iter:  4 | assem : 3.96s | CG took 1.51s with   98 iterations | alpha : 1.00 | energy = -1.6343877879 | relres = 9.75e-03 |
Iter:  5 | assem : 3.90s | CG took 0.65s with   43 iterations | alpha : 0.50 | energy = -1.6434091644 | relres = 2.79e-03 |
Iter:  6 | assem : 3.89s | CG took 0.63s with   40 iterations | alpha : 1.00 | energy = -1.6449128904 | relres = 1.30e-03 |
Iter:  7 | assem : 3.97s | CG took 0.96s with   60 iterations | alpha : 0.50 | energy = -1.6462055819 | relres = 4.17e-04 |
Iter:  8 | assem : 4.08s | CG took 0.43s with   27 it

In [6]:
L2 = ngs.VectorL2(mesh, dim = 3, order=2)
Bl2 = ngs.GridFunction(L2)
with ngs.TaskManager(): Bl2.Set(B)
Bl2np = Bl2.vec.FV().NumPy()
HCurl.ndof

import matplotlib.pyplot as plt
import numpy as np
max(abs(Bl2np))

17.92496214668886

In [7]:
Draw(B, mesh, clipping={"z":-1, "dist":0.064})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [8]:
help(CGSolver)

Help on class CGSolver in module ngsolve.krylovspace:

class CGSolver(LinearSolver)
 |  CGSolver(*args, conjugate: bool = False, abstol: float = None, maxsteps: int = None, printing: bool = False, **kwargs)
 |  
 |  Method resolution order:
 |      CGSolver
 |      LinearSolver
 |      ngsolve.la.BaseMatrix
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, conjugate: bool = False, abstol: float = None, maxsteps: int = None, printing: bool = False, **kwargs)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: ngsolve.la.BaseMatrix) -> None
 |      
 |      2. __init__(self: ngsolve.la.BaseMatrix, arg0: ngsolve.la.BaseVector) -> None
 |      
 |      3. __init__(self: ngsolve.la.BaseMatrix, arg0: ngsolve.la.MultiVector) -> None
 |      
 |      4. __init__(self: ngsolve.la.BaseMatrix, arg0: ngsolve.bla.MatrixD) -> None
 |      
 |      5. __init__(self: ngsolve.la.B